<a href="https://colab.research.google.com/github/Janani-selvakumaravelan/SmartBot-NLP/blob/main/AI_Chatbot_A_Conversational_Assistant_using_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install nltk tensorflow numpy


In [ ]:
import json
import numpy as np
import nltk
from nltk.stem import PorterStemmer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
import random
import pickle

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('punkt_tab') # Download punkt_tab for sentence tokenization


stemmer = PorterStemmer()

with open('intents.json') as file:
    data = json.load(file)

words = []
labels = []
docs_x = []
docs_y = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent['tag'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

words = [stemmer.stem(w.lower()) for w in words if w not in ['?', '.', '!']]
words = sorted(list(set(words)))
labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []
    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        bag.append(1 if w in wrds else 0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

training = np.array(training)
output = np.array(output)

# Save data
with open('data.pickle', 'wb') as f:
    pickle.dump((words, labels, training, output), f)

# Model
model = Sequential()
model.add(Dense(128, input_shape=(len(training[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(output[0]), activation='softmax'))

sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(training, output, epochs=200, batch_size=5, verbose=1)
model.save('chat_model.h5')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Epoch 1/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.3917 - loss: 1.5490
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3167 - loss: 1.4507
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4583 - loss: 1.5009
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2667 - loss: 1.4441
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2917 - loss: 1.5070
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3750 - loss: 1.4716
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2583 - loss: 1.4686
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6167 - loss: 1.2802
Epoch 9/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5083 - loss: 1.3259
Epoch 10/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6000 - loss: 1.1743
Epoch 11/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5667 - loss: 1.2640
Epoch 12/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5167 - lo

In [ ]:
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import PorterStemmer
from tensorflow.keras.models import load_model

stemmer = PorterStemmer()

nltk.download('punkt')

with open('intents.json') as file:
    data = json.load(file)

with open('data.pickle', 'rb') as f:
    words, labels, training, output = pickle.load(f)

model = load_model('chat_model.h5')

def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return np.array(bag)

def chat():
    print("Start chatting with the bot (type 'quit' to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict(np.array([bag_of_words(inp, words)]))[0]
        results_index = np.argmax(results)
        tag = labels[results_index]

        if results[results_index] > 0.7:
            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']
            print("Bot:", random.choice(responses))
        else:
            print("Bot: Sorry, I didn't understand that.")

chat()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Start chatting with the bot (type 'quit' to stop)!
You: hi
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
Bot: How can I assist you today?
You: hi
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Bot: How can I assist you today?
You: hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Bot: Hello!
You: good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Bot: How can I assist you today?
You: thanks
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Bot: Glad I could help!
You: hours
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Bot: We are open 9 AM to 5 PM, Monday through Friday.
You: noanswer
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Bot: Sorry, I didn't understand that.
You: noanswe
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Bot: Sorry, I didn't understand that.
You: noanswer
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Bot: Sorry, I didn't understand that.
You: See you later
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Bot: Have a great day!
You: Is there anyone?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Bot: Hello!
You: Is anyone there?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 3